In [23]:
import numpy as np
import pandas as pd
from munkres import Munkres,print_matrix

In [34]:
def initImport(rd):
    
    global has_repeated
    has_repeated = {}
    has_repeated["same_do-opp"] = {}
    has_repeated["opp_do-same"] = {}
    has_repeated["opp_dt-opp"] = {}
    has_repeated["opp_do-opp"] = {}


    global conflicts
    conflicts = {}

    global teams
    teams = []
    global sides
    sides = {}
    global dts #dts = Destination Teams
    dts = {}
    global dos #dos = Destination Oppositions
    dos = {}
    data_csv = pd.read_csv("data_afterr{}.csv".format(rd),index_col=0)
    conflicts_csv = pd.read_csv("conflicts.csv",index_col=0)

    # Create empty conflict dictionaries
    for ind,row in data_csv.iterrows():
        teams.append(ind)
        sides[ind] = {}
        sides[ind][1] = row[1]
        sides[ind][2] = row[4]
        sides[ind][3] = row[7]
    
        dts[ind] = {}
        dts[ind][1] = row[2]
        dts[ind][2] = row[5]
        dts[ind][3] = row[8]
    
        dos[ind] = {}
        dos[ind][1] = row[3]
        dos[ind][2] = row[6]
        dos[ind][3] = row[9]
    
    
    
        conflicts["{}_p".format(ind)] = []
        conflicts["{}_d".format(ind)] = []

    conflicts_csv.head()
    for ind,row in conflicts_csv.iterrows():
        team1 = ind
        team2 = row[0]
        
        if (team1 != "nan") and (team2 != "nan"):
            #1p, 2p
            if "{}_p".format(team2) not in conflicts["{}_p".format(team1)]:
                conflicts["{}_p".format(team1)].append("{}_p".format(team2))
            #1d, 2p
            if "{}_p".format(team2) not in conflicts["{}_d".format(team1)]:
                conflicts["{}_d".format(team1)].append("{}_p".format(team2))
            #1p, 2d
            if "{}_d".format(team2) not in conflicts["{}_p".format(team1)]:
                conflicts["{}_p".format(team1)].append("{}_d".format(team2))
            #1d, 2d
            if "{}_d".format(team2) not in conflicts["{}_d".format(team1)]:
                conflicts["{}_d".format(team1)].append("{}_d".format(team2))
            
            #2p, 1p
            if "{}_p".format(team1) not in conflicts["{}_p".format(team2)]:
                conflicts["{}_p".format(team2)].append("{}_p".format(team1))
            #2d, 1p
            if "{}_p".format(team1) not in conflicts["{}_d".format(team2)]:
                conflicts["{}_d".format(team2)].append("{}_p".format(team1))
            #2p, 1d
            if "{}_d".format(team1) not in conflicts["{}_p".format(team2)]:
                conflicts["{}_p".format(team2)].append("{}_d".format(team1))
            #2d, 1d
            if "{}_d".format(team1) not in conflicts["{}_d".format(team2)]:
                conflicts["{}_d".format(team2)].append("{}_d".format(team1))
        #print(conflicts)
        
def addConflicts(rd):
    for team in teams:
        
        if rd >= 2:
            # Destination Team
            if dts[team][1] not in conflicts["{}_{}".format(team,sides[team][1])]:
                conflicts["{}_{}".format(team,sides[team][1])].append(dts[team][1])
                    
            # Destination's Opponent
            if dos[team][1] not in conflicts["{}_{}".format(team,sides[team][1])]:
                conflicts["{}_{}".format(team,sides[team][1])].append(dos[team][1])
            
        if rd >= 3:
            # Destination Team
            if dts[team][2] not in conflicts["{}_{}".format(team,sides[team][2])]:
                conflicts["{}_{}".format(team,sides[team][2])].append(dts[team][2])
                    
            # Destination's Opponent
            if dos[team][2] not in conflicts["{}_{}".format(team,sides[team][2])]:
                conflicts["{}_{}".format(team,sides[team][2])].append(dos[team][2])
     
        if rd >= 4:
            # Destination Team
            if dts[team][3] not in conflicts["{}_{}".format(team,sides[team][3])]:
                conflicts["{}_{}".format(team,sides[team][3])].append(dts[team][3])
                    
            # Destination's Opponent
            if dos[team][3] not in conflicts["{}_{}".format(team,sides[team][3])]:
                conflicts["{}_{}".format(team,sides[team][3])].append(dos[team][3])
            
def generateCosts(rd):
    global masterpairings
    global pairings
    global roomranks
    global costs
    global softconflictcosts
    global powercosts
    global cannotgoto
    global sendtorooms
    global receivefromrooms
    global venues_d1
    global venues_d2
    
    cannotgoto = {}
    sendtorooms = {}
    receivefromrooms = {}
    
    masterpairings = {}
    pairings = {}
    pairings[1] = {}
    pairings[2] = {}
    roomranks = {}
    costs = {}
    softconflictcosts = {}
    powercosts = {}
    venuesbyteam = {}
    
    venues_d1 = {}
    venues_d2 = {}
    
    div1pairings_csv = pd.read_csv("pairings_r{}_div1.csv".format(rd),index_col=0)
    div2pairings_csv = pd.read_csv("pairings_r{}_div2.csv".format(rd),index_col=0)
    
    index1 = 0
    for ind,row in div1pairings_csv.iterrows():
        pairings[1][row[0]] = [row[1],row[2]]
        masterpairings[row[0]] = [row[1],row[2]]
        roomranks[row[0]] = ind
        venuesbyteam[row[1]] = row[0]
        venuesbyteam[row[2]] = row[0]
        cannotgoto[row[1]] = []
        cannotgoto[row[2]] = []
        sendtorooms[row[0]] = []
        receivefromrooms[row[0]] = []
        
        venues_d1[index1] = row[0]
        index1 += 1
    
    index2 = 0
    for ind,row in div2pairings_csv.iterrows():
        pairings[2][row[0]] = [row[1],row[2]]
        masterpairings[row[0]] = [row[1],row[2]]
        roomranks[row[0]] = ind
        venuesbyteam[row[1]] = row[0]
        venuesbyteam[row[2]] = row[0]
        cannotgoto[row[1]] = []
        cannotgoto[row[2]] = []
        sendtorooms[row[0]] = []
        receivefromrooms[row[0]] = []
        
        venues_d2[index2] = row[0]
        index2 += 1

    #print(masterpairings)
    for div1room,div1pairing in pairings[1].items():
        for div2room,div2pairing in pairings[2].items():
            costs[div1room,div2room] = 0
            costs[div2room,div1room] = 0
            
            softconflictcosts[div1room,div2room] = 0
            softconflictcosts[div2room,div1room] = 0
            
            powercosts[div1room,div2room] = 0
            powercosts[div2room,div1room] = 0

    for assignment,cost in costs.items():
        originPairing = masterpairings[assignment[0]]
        destinationPairing = masterpairings[assignment[1]]
        
        costs[assignment] += 100*np.absolute(roomranks[assignment[0]] - roomranks[assignment[1]])
        powercosts[assignment] += 100*np.absolute(roomranks[assignment[0]] - roomranks[assignment[1]])
    
        # Conflicts
        if destinationPairing[0] in conflicts["{}_p".format(originPairing[0])]:
            costs[assignment] = 99999999999
            cannotgoto[originPairing[0]].append(venuesbyteam[destinationPairing[0]])
        if destinationPairing[0] in conflicts["{}_d".format(originPairing[1])]:
            costs[assignment] = 99999999999
            cannotgoto[originPairing[1]].append(venuesbyteam[destinationPairing[0]])
        
        # Can't send if the destination D team is in the P/D team's conflicts
        if destinationPairing[1] in conflicts["{}_p".format(originPairing[0])]:
            costs[assignment] = 99999999999
            cannotgoto[originPairing[0]].append(venuesbyteam[destinationPairing[1]])
        if destinationPairing[1] in conflicts["{}_d".format(originPairing[1])]:
            costs[assignment] = 99999999999
            cannotgoto[originPairing[1]].append(venuesbyteam[destinationPairing[1]])
            
              
def allocate(rd):
    
    # Form the matrix
    # Division 1 - Rows are Div 1 rooms, Columns are Div 2 rooms
    div1matrix = []
    for rkey,rval in venues_d1.items():
        row = []
        for ckey,cval in venues_d2.items():
            row.append(costs[rval,cval])
        
        div1matrix.append(row)

    
    # Division 2 - Rows are Div 2 rooms, Columns are Div 1 rooms
    div2matrix = []
    for rkey,rval in venues_d2.items():
        row = []
        for ckey,cval in venues_d1.items():
            row.append(costs[rval,cval])
        
        div2matrix.append(row)
    
    # Division 1 Munkres
    m1 = Munkres()
    i1 = m1.compute(div1matrix)
    #print_matrix(div1matrix,msg="Optimising this Division 1 matrix")
    total1 = 0
    for r,c in i1:
        value = div1matrix[r][c]
        total1 += value
        print("{},{} -> {}".format(venues_d1[r],venues_d2[c],value))
    print("Total Cost: {}".format(total1))
    
    print("---")
    
    # Division 2 Munkres
    m2 = Munkres()
    i2 = m2.compute(div2matrix)
    #print_matrix(div2matrix,msg="Optimising this Division 2 matrix")
    total2 = 0
    for r, c in i2:
        value = div2matrix[r][c]
        total2 += value
        print("{},{} -> {}".format(venues_d2[r],venues_d1[c],value))
    
    print("Total Cost: {}".format(total2))   

def manualConflict(a,b):
    costs[(a,b)] = 99999999999

In [37]:
nextRound = 4
lastRound = nextRound - 1

In [38]:
initImport(lastRound)
addConflicts(nextRound)
generateCosts(nextRound)
#print(costs)

# Add manual conflicts (reciprocate)
manualConflict("15A","15D")
manualConflict("15D","15A")
manualConflict("15D","15A")
manualConflict("9B","15D")
manualConflict("15D","9B")
manualConflict("12A","21D")
manualConflict("21D","12A")

#print(costs)
allocate(nextRound)

14D,21D -> 200
14A,18B -> 100
11A,21C -> 300
12A,18C -> 0
9B,20D -> 0
14B,20A -> 100
15A,18A -> 500
11D,15D -> 0
15C,20C -> 0
11B,18D -> 0
Total Cost: 1200
---
18B,14A -> 100
18A,11A -> 100
21D,14D -> 200
18C,9B -> 100
20D,12A -> 100
21C,15A -> 100
20A,14B -> 100
15D,11D -> 0
20C,15C -> 0
18D,11B -> 0
Total Cost: 800
